In [1]:
%%bash
chia version
cdv --version

1.4.0
cdv, version 1.0.8


# Tic Tac Toe

Inner Puzzle is [tic tac toe coin](tic-tac-toe-coin.ipynb).

In [2]:
from pathlib import Path
import sys

# import singleton_helpers and tic_tac_toe
sys.path.insert(0, ".")
import tic_tac_toe
sys.path.insert(0, "..")
import singleton_helpers

In [3]:
from chia.types.blockchain_format.program import Program
from clvm_tools.clvmc import compile_clvm_text
def load_program(file_path, search_paths):
    clsp = Path(file_path).read_text()
    return Program(
        compile_clvm_text(clsp, search_paths)
    )

tic_tac_toe_puzzle = load_program('tic-tac-toe.clsp', '.')
tic_tac_toe_coin_puzzle = load_program('tic-tac-toe-coin.clsp', '.')

In [4]:
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)
from cdv.test import Network, Wallet

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
bob: Wallet = network.make_wallet("bob")
await network.farm_block(farmer=alice)
await network.farm_block(farmer=bob)

print(f'alice balance:\t\t{alice.balance()}')
print(f'alice puzzle hash:\t{alice.puzzle_hash}')
print(f'bob balance:\t\t{bob.balance()}')
print(f'bob puzzle hash:\t{bob.puzzle_hash}')

alice balance:		2000000000000
alice puzzle hash:	4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3
bob balance:		2000000000000
bob puzzle hash:	87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5


In [5]:
# prepare tic tac toe coin
# empty board
board = [0] * 9
# player one (x)
player = 1
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(tic_tac_toe_puzzle, board, player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    alice.puzzle_hash, # player one (x)
    bob.puzzle_hash, # player two (o)
    tic_tac_toe_puzzle, 
    curried_tic_tac_toe_puzzle)

In [6]:
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

inner_puzzle = singleton_top_layer.adapt_inner_to_singleton(curried_tic_tac_toe_coin_puzzle)
standard_coin_message, coin_spends = singleton_helpers.get_create_singleton_coin_spends(
                    standard_txn_coin,
                    standard_txn_puzzle,
                    odd_amount = 1023, 
                    inner_puzzle = inner_puzzle,
                    keys_values = [
                        ("game", "tic tac toe"),
                        ("player one", "alice"),
                        ("player two", "bob")
                    ]
              )
standard_txn_coin_spend = coin_spends[0]
launcher_coin_spend = coin_spends[1]
print("\nstarting coin spend:")
singleton_helpers.print_json(standard_txn_coin_spend.to_json_dict())
print("\nlauncher coin spend:")
singleton_helpers.print_json(launcher_coin_spend.to_json_dict())
print(f'\nlauncher id: {launcher_coin_spend.coin.name()}')
print(f'\nstandard_coin_message: {standard_coin_message.hex()}')


starting coin spend:
{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff8203ff80ffff3dffa0

In [7]:
# sign the standard txn spend
from blspy import AugSchemeMPL, PrivateKey
from chia.consensus.default_constants import DEFAULT_CONSTANTS

synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)

signature = AugSchemeMPL.sign(synthetic_sk,standard_coin_message)

# push txn to create an eve singleton
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    coin_spends,
    signature
)
await network.push_tx(spend_bundle)

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>, amount=1023),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1749999998977),
  Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: eb7c67f3be28d13d0f05340990786fb910ff8205ca574914fc554ac5c9eb2f33>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c06

In [8]:
from clvm_tools.binutils import disassemble
# player 1 plays position 4
position = 4
print(f'{board} {player} {position}')

solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

[0, 0, 0, 0, 0, 0, 0, 0, 0] 1 4
((73 1023) (51 0xed945684c8cc676792f5720a52830ad64c72367fcca8f33c74dd6c9110d0e153 1023))


In [10]:
## verify puzzle
next_player = 2
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(
    tic_tac_toe_puzzle, 
    board, next_player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    alice.puzzle_hash,
    bob.puzzle_hash,
    tic_tac_toe_puzzle, 
    curried_tic_tac_toe_puzzle)

   |   |   
---+---+---
   | x |   
---+---+---
   |   |   


In [11]:
# spend eve singleton
network.sim.pass_blocks(1)
launcher_id = launcher_coin_spend.coin.name()
singleton_struct = (
        singleton_top_layer.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer.SINGLETON_LAUNCHER_HASH)
    )

singleton_puzzle = singleton_top_layer.SINGLETON_MOD.curry(
            singleton_struct,
            inner_puzzle,
    )

singleton_coin = await singleton_helpers.get_unspent_singleton(
    get_coin_records_by_parent_ids, 
    launcher_id) 

lineage_proof = singleton_top_layer.lineage_proof_for_coinsol(launcher_coin_spend)

current_value = 0 # eve singleton has zero value
inner_solution = Program.to([
    1023, 
    position
])

singleton_coin_spend = singleton_helpers.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
    )

print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}]')

print('\nsingleton_coin_spend:')
singleton_helpers.print_json(singleton_coin_spend.to_json_dict())

singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570',
 'puzzle_hash': '0xeb7c67f3be28d13d0f05340990786fb910ff8205ca574914fc554ac5c9eb2f33'}

lineage_proof:
{'amount': 1023,
 'inner_puzzle_hash': None,
 'parent_name': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba'}

inner_solution:
ff8203ffff0480]

singleton_coin_spend:
{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570",
        "puzzle_hash": "0xeb7c67f3be28d13d0f05340990786fb910ff8205ca574914fc554ac5c9eb2f33"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff18ff2fffff010180ffff01ff02ff36ffff04ff02ffff04ff05ffff04ff17ffff04ffff02ff26ffff04ff02ffff04ff0bff80808080ffff04ff2fffff04ff0bffff04ff5fff808080808080808080ffff01ff088080ff0180ffff04ffff01ffffffff4602ff3304ffff0101ff02ffff02ffff03ff05ffff01ff02ff5cffff04ff02ffff04ff0dffff04ffff0bff2cffff0bff

In [ ]:
%%bash
opd ff02ffff01ff02ffff01ff02ffff03ffff18ff2fffff010180ffff01ff02ff36ffff04ff02ffff04ff05ffff04ff17ffff04ffff02ff26ffff04ff02ffff04ff0bff80808080ffff04ff2fffff04ff0bffff04ff5fff808080808080808080ffff01ff088080ff0180ffff04ffff01ffffffff4602ff3304ffff0101ff02ffff02ffff03ff05ffff01ff02ff5cffff04ff02ffff04ff0dffff04ffff0bff2cffff0bff24ff3880ffff0bff2cffff0bff2cffff0bff24ff3480ff0980ffff0bff2cff0bffff0bff24ff8080808080ff8080808080ffff010b80ff0180ff02ffff03ff0bffff01ff02ff32ffff04ff02ffff04ff05ffff04ff0bffff04ff17ffff04ffff02ff2affff04ff02ffff04ffff02ffff03ffff09ff23ff2880ffff0181b3ff8080ff0180ff80808080ff80808080808080ffff01ff02ffff03ff17ff80ffff01ff088080ff018080ff0180ffffffff0bffff0bff17ffff02ff3affff04ff02ffff04ff09ffff04ff2fffff04ffff02ff26ffff04ff02ffff04ff05ff80808080ff808080808080ff5f80ff0bff81bf80ff02ffff03ffff20ffff22ff4fff178080ffff01ff02ff7effff04ff02ffff04ff6fffff04ffff04ffff02ffff03ff4fffff01ff04ff23ffff04ffff02ff3affff04ff02ffff04ff09ffff04ff53ffff04ffff02ff26ffff04ff02ffff04ff05ff80808080ff808080808080ffff04ff81b3ff80808080ffff011380ff0180ffff02ff7cffff04ff02ffff04ff05ffff04ff1bffff04ffff21ff4fff1780ff80808080808080ff8080808080ffff01ff088080ff0180ffff04ffff09ffff18ff05ffff010180ffff010180ffff09ff05ffff01818f8080ff0bff2cffff0bff24ff3080ffff0bff2cffff0bff2cffff0bff24ff3480ff0580ffff0bff2cffff02ff5cffff04ff02ffff04ff07ffff04ffff0bff24ff2480ff8080808080ffff0bff24ff8080808080ffffff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff26ffff04ff02ffff04ff09ff80808080ffff02ff26ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff02ff5effff04ff02ffff04ff05ffff04ff0bffff04ffff02ff3affff04ff02ffff04ff09ffff04ff17ffff04ffff02ff26ffff04ff02ffff04ff05ff80808080ff808080808080ffff04ff17ffff04ff2fffff04ff5fffff04ff81bfff80808080808080808080ffff04ffff04ff20ffff04ff17ff808080ffff02ff7cffff04ff02ffff04ff05ffff04ffff02ff82017fffff04ffff04ffff04ff17ff2f80ffff04ffff04ff5fff81bf80ffff04ff0bff05808080ff8202ff8080ffff01ff80808080808080ffff02ff2effff04ff02ffff04ff05ffff04ff0bffff04ffff02ffff03ff3bffff01ff02ff22ffff04ff02ffff04ff05ffff04ff17ffff04ff13ffff04ff2bffff04ff5bffff04ff5fff808080808080808080ffff01ff02ffff03ffff09ff15ffff0bff13ff1dff2b8080ffff01ff0bff15ff17ff5f80ffff01ff088080ff018080ff0180ffff04ff17ffff04ff2fffff04ff5fffff04ff81bfffff04ff82017fff8080808080808080808080ff02ffff03ff05ffff011bffff010b80ff0180ff018080ffff04ffff01ffa024e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fdffa06a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff04ffff01ff02ffff01ff02ffff01ff02ffff01ff02ff1cffff04ff02ffff04ff05ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff8202ffffff04ffff02ff5fffff04ff82017fff808080ff80808080808080808080ffff04ffff01ffff49ff33ff02ffff03ffff20ffff09ff80ff82027f8080ffff01ff04ffff04ff14ffff01ff84900ddeedff818f8080ffff02ffff03ffff09ffff0181ffff82027f80ff80ffff01ff04ffff02ffff03ffff09ffff0101ff82027f80ffff01ff04ff14ffff04ff0bffff04ff5fff80808080ffff01ff04ff14ffff04ff17ffff04ff5fff8080808080ff0180ff808080ff018080ffff01ff02ff12ffff04ff02ffff04ffff02ff1effff04ff02ffff04ffff02ff1affff04ff02ffff04ff05ffff04ffff04ff05ffff04ff0bffff04ff17ffff04ff2fffff04ffff02ff1affff04ff02ffff04ff2fffff04ffff04ff82057fffff04ff81bfff808080ff8080808080ff808080808080ff8080808080ff80808080ffff04ff5fff808080808080ff0180ffffff04ffff04ff08ffff04ff0bff808080ffff04ffff04ff14ffff04ff05ffff04ff0bff80808080ff808080ff04ffff0102ffff04ffff04ffff0101ff0580ffff04ffff02ff16ffff04ff02ffff04ff0bffff01ff0180808080ff80808080ffff02ffff03ff05ffff01ff04ffff0104ffff04ffff04ffff0101ff0980ffff04ffff02ff16ffff04ff02ffff04ff0dffff04ff0bff8080808080ff80808080ffff010b80ff0180ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff1effff04ff02ffff04ff09ff80808080ffff02ff1effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01ff02ffff01ff02ff1cffff04ff02ffff04ff05ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff8202ffffff04ffff02ff5fffff04ff82017fff808080ff80808080808080808080ffff04ffff01ffff49ff33ff02ffff03ffff20ffff09ff80ff82027f8080ffff01ff04ffff04ff14ffff01ff84900ddeedff818f8080ffff02ffff03ffff09ffff0181ffff82027f80ff80ffff01ff04ffff02ffff03ffff09ffff0101ff82027f80ffff01ff04ff14ffff04ff0bffff04ff5fff80808080ffff01ff04ff14ffff04ff17ffff04ff5fff8080808080ff0180ff808080ff018080ffff01ff02ff12ffff04ff02ffff04ffff02ff1effff04ff02ffff04ffff02ff1affff04ff02ffff04ff05ffff04ffff04ff05ffff04ff0bffff04ff17ffff04ff2fffff04ffff02ff1affff04ff02ffff04ff2fffff04ffff04ff82057fffff04ff81bfff808080ff8080808080ff808080808080ff8080808080ff80808080ffff04ff5fff808080808080ff0180ffffff04ffff04ff08ffff04ff0bff808080ffff04ffff04ff14ffff04ff05ffff04ff0bff80808080ff808080ff04ffff0102ffff04ffff04ffff0101ff0580ffff04ffff02ff16ffff04ff02ffff04ff0bffff01ff0180808080ff80808080ffff02ffff03ff05ffff01ff04ffff0104ffff04ffff04ffff0101ff0980ffff04ffff02ff16ffff04ff02ffff04ff0dffff04ff0bff8080808080ff80808080ffff010b80ff0180ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff1effff04ff02ffff04ff09ff80808080ffff02ff1effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01a04f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3ffff04ffff01a087908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5ffff04ffff01ff02ffff01ff02ff0affff04ff02ffff04ffff02ff1effff04ff02ffff04ff05ffff04ff0bffff04ff80ffff04ff17ff80808080808080ffff04ff0bff8080808080ffff04ffff01ffffff02ffff03ffff22ffff20ffff09ff09ff808080ffff20ffff09ff15ff808080ffff20ffff09ff2dff808080ffff20ffff09ff5dff808080ffff20ffff09ff81bdff808080ffff20ffff09ff82017dff808080ffff20ffff09ff8202fdff808080ffff20ffff09ff8205fdff808080ffff20ffff09ff820bfdff80808080ffff01ff0181ffffff01ff02ff16ffff04ff02ffff04ffff04ff09ffff04ff15ffff04ff2dffff04ff5dffff04ff81bdffff04ff82017dffff04ff8202fdffff04ff8205fdffff04ff820bfdff80808080808080808080ffff04ff0bff808080808080ff0180ff02ffff03ffff22ffff09ff05ff2f80ffff09ff0bff2f80ffff09ff17ff2f8080ffff01ff0101ff8080ff0180ffff04ffff02ff08ffff04ff02ffff04ff05ffff04ff0bff8080808080ffff04ff05ff808080ffff02ffff03ffff21ffff02ff0cffff04ff02ffff04ff09ffff04ff15ffff04ff2dffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff5dffff04ff81bdffff04ff82017dffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff8202fdffff04ff8205fdffff04ff820bfdffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff09ffff04ff5dffff04ff8202fdffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff15ffff04ff81bdffff04ff8205fdffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff2dffff04ff82017dffff04ff820bfdffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff09ffff04ff81bdffff04ff820bfdffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff2dffff04ff81bdffff04ff8202fdffff04ff0bff8080808080808080ffff010bff8080ff0180ff02ffff03ffff07ff0580ffff01ff04ffff02ffff03ffff09ff17ff2f80ffff01ff02ffff03ffff09ff80ff0980ffff010bffff01ff08ffff018c757064617465206572726f728080ff0180ffff010980ff0180ffff02ff1effff04ff02ffff04ff0dffff04ff0bffff04ffff10ff17ffff010180ffff04ff2fff8080808080808080ff8080ff0180ff018080ffff04ffff01ff02ffff01ff02ffff01ff02ff0affff04ff02ffff04ffff02ff1effff04ff02ffff04ff05ffff04ff0bffff04ff80ffff04ff17ff80808080808080ffff04ff0bff8080808080ffff04ffff01ffffff02ffff03ffff22ffff20ffff09ff09ff808080ffff20ffff09ff15ff808080ffff20ffff09ff2dff808080ffff20ffff09ff5dff808080ffff20ffff09ff81bdff808080ffff20ffff09ff82017dff808080ffff20ffff09ff8202fdff808080ffff20ffff09ff8205fdff808080ffff20ffff09ff820bfdff80808080ffff01ff0181ffffff01ff02ff16ffff04ff02ffff04ffff04ff09ffff04ff15ffff04ff2dffff04ff5dffff04ff81bdffff04ff82017dffff04ff8202fdffff04ff8205fdffff04ff820bfdff80808080808080808080ffff04ff0bff808080808080ff0180ff02ffff03ffff22ffff09ff05ff2f80ffff09ff0bff2f80ffff09ff17ff2f8080ffff01ff0101ff8080ff0180ffff04ffff02ff08ffff04ff02ffff04ff05ffff04ff0bff8080808080ffff04ff05ff808080ffff02ffff03ffff21ffff02ff0cffff04ff02ffff04ff09ffff04ff15ffff04ff2dffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff5dffff04ff81bdffff04ff82017dffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff8202fdffff04ff8205fdffff04ff820bfdffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff09ffff04ff5dffff04ff8202fdffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff15ffff04ff81bdffff04ff8205fdffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff2dffff04ff82017dffff04ff820bfdffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff09ffff04ff81bdffff04ff820bfdffff04ff0bff80808080808080ffff02ff0cffff04ff02ffff04ff2dffff04ff81bdffff04ff8202fdffff04ff0bff8080808080808080ffff010bff8080ff0180ff02ffff03ffff07ff0580ffff01ff04ffff02ffff03ffff09ff17ff2f80ffff01ff02ffff03ffff09ff80ff0980ffff010bffff01ff08ffff018c757064617465206572726f728080ff0180ffff010980ff0180ffff02ff1effff04ff02ffff04ff0dffff04ff0bffff04ffff10ff17ffff010180ffff04ff2fff8080808080808080ff8080ff0180ff018080ffff04ffff01ff80ff80ff80ff80ff80ff80ff80ff80ff8080ffff04ffff0101ff01808080ff01808080808080ffff06ff018080ff01808080

(a (q 2 (q 2 (i (logand 47 (q . 1)) (q 2 54 (c 2 (c 5 (c 23 (c (a 38 (c 2 (c 11 ()))) (c 47 (c 11 (c 95 ())))))))) (q 8)) 1) (c (q (((70 . 2) 51 . 4) (q . 1) 2 (a (i 5 (q 2 92 (c 2 (c 13 (c (sha256 44 (sha256 36 56) (sha256 44 (sha256 44 (sha256 36 52) 9) (sha256 44 11 (sha256 36 ())))) ())))) (q . 11)) 1) 2 (i 11 (q 2 50 (c 2 (c 5 (c 11 (c 23 (c (a 42 (c 2 (c (a (i (= 35 40) (q . -77) ()) 1) ()))) ())))))) (q 2 (i 23 () (q 8)) 1)) 1) (((sha256 (sha256 23 (a 58 (c 2 (c 9 (c 47 (c (a 38 (c 2 (c 5 ()))) ()))))) 95) 11 -65) 2 (i (not (all 79 23)) (q 2 126 (c 2 (c 111 (c (c (a (i 79 (q 4 35 (c (a 58 (c 2 (c 9 (c 83 (c (a 38 (c 2 (c 5 ()))) ()))))) (c -77 ()))) (q . 19)) 1) (a 124 (c 2 (c 5 (c 27 (c (any 79 23) ())))))) ())))) (q 8)) 1) (c (= (logand 5 (q . 1)) (q . 1)) (= 5 (q . -113))) 11 44 (sha256 36 48) (sha256 44 (sha256 44 (sha256 36 52) 5) (sha256 44 (a 92 (c 2 (c 7 (c (sha256 36 36) ())))) (sha256 36 ())))) ((a (i (l 5) (q 11 (q . 2) (a 38 (c 2 (c 9 ()))) (a 38 (c 2 (c 13 ())))) (q

In [ ]:
%%bash
opd ffffa012d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bbaff8203ff80ff8203ffffff8203ffff04ff028080

((0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba 1023) 1023 (1023 4 2))


In [ ]:
%%bash
brun '(a (q 2 (q 2 (i (logand 47 (q . 1)) (q 2 54 (c 2 (c 5 (c 23 (c (a 38 (c 2 (c 11 ()))) (c 47 (c 11 (c 95 ())))))))) (q 8)) 1) (c (q (((70 . 2) 51 . 4) (q . 1) 2 (a (i 5 (q 2 92 (c 2 (c 13 (c (sha256 44 (sha256 36 56) (sha256 44 (sha256 44 (sha256 36 52) 9) (sha256 44 11 (sha256 36 ())))) ())))) (q . 11)) 1) 2 (i 11 (q 2 50 (c 2 (c 5 (c 11 (c 23 (c (a 42 (c 2 (c (a (i (= 35 40) (q . -77) ()) 1) ()))) ())))))) (q 2 (i 23 () (q 8)) 1)) 1) (((sha256 (sha256 23 (a 58 (c 2 (c 9 (c 47 (c (a 38 (c 2 (c 5 ()))) ()))))) 95) 11 -65) 2 (i (not (all 79 23)) (q 2 126 (c 2 (c 111 (c (c (a (i 79 (q 4 35 (c (a 58 (c 2 (c 9 (c 83 (c (a 38 (c 2 (c 5 ()))) ()))))) (c -77 ()))) (q . 19)) 1) (a 124 (c 2 (c 5 (c 27 (c (any 79 23) ())))))) ())))) (q 8)) 1) (c (= (logand 5 (q . 1)) (q . 1)) (= 5 (q . -113))) 11 44 (sha256 36 48) (sha256 44 (sha256 44 (sha256 36 52) 5) (sha256 44 (a 92 (c 2 (c 7 (c (sha256 36 36) ())))) (sha256 36 ())))) ((a (i (l 5) (q 11 (q . 2) (a 38 (c 2 (c 9 ()))) (a 38 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 2 94 (c 2 (c 5 (c 11 (c (a 58 (c 2 (c 9 (c 23 (c (a 38 (c 2 (c 5 ()))) ()))))) (c 23 (c 47 (c 95 (c -65 ()))))))))) (c (c 32 (c 23 ())) (a 124 (c 2 (c 5 (c (a 383 (c (c (c 23 47) (c (c 95 -65) (c 11 5))) 767)) (q ())))))) (a 46 (c 2 (c 5 (c 11 (c (a (i 59 (q 2 34 (c 2 (c 5 (c 23 (c 19 (c 43 (c 91 (c 95 ())))))))) (q 2 (i (= 21 (sha256 19 29 43)) (q 11 21 23 95) (q 8)) 1)) 1) (c 23 (c 47 (c 95 (c -65 (c 383 ())))))))))) 2 (i 5 (q . 27) (q . 11)) 1) 1)) (c (q 0x24e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fd 0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) (c (q 2 (q 2 (q 2 (q 2 28 (c 2 (c 5 (c 11 (c 23 (c 47 (c -65 (c 767 (c (a 95 (c 383 ())) ()))))))))) (c (q (73 51 2 (i (not (= () 639)) (q 4 (c 20 (q 0x900ddeed -113)) (a (i (= (q . -1) 639) () (q 4 (a (i (= (q . 1) 639) (q 4 20 (c 11 (c 95 ()))) (q 4 20 (c 23 (c 95 ())))) 1) ())) 1)) (q 2 18 (c 2 (c (a 30 (c 2 (c (a 26 (c 2 (c 5 (c (c 5 (c 11 (c 23 (c 47 (c (a 26 (c 2 (c 47 (c (c 1407 (c -65 ())) ())))) ()))))) ())))) ()))) (c 95 ()))))) 1) ((c (c 8 (c 11 ())) (c (c 20 (c 5 (c 11 ()))) ())) 4 (q . 2) (c (c (q . 1) 5) (c (a 22 (c 2 (c 11 (q 1)))) ()))) (a (i 5 (q 4 (q . 4) (c (c (q . 1) 9) (c (a 22 (c 2 (c 13 (c 11 ())))) ()))) (q . 11)) 1) 2 (i (l 5) (q 11 (q . 2) (a 30 (c 2 (c 9 ()))) (a 30 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q 2 (q 2 28 (c 2 (c 5 (c 11 (c 23 (c 47 (c -65 (c 767 (c (a 95 (c 383 ())) ()))))))))) (c (q (73 51 2 (i (not (= () 639)) (q 4 (c 20 (q 0x900ddeed -113)) (a (i (= (q . -1) 639) () (q 4 (a (i (= (q . 1) 639) (q 4 20 (c 11 (c 95 ()))) (q 4 20 (c 23 (c 95 ())))) 1) ())) 1)) (q 2 18 (c 2 (c (a 30 (c 2 (c (a 26 (c 2 (c 5 (c (c 5 (c 11 (c 23 (c 47 (c (a 26 (c 2 (c 47 (c (c 1407 (c -65 ())) ())))) ()))))) ())))) ()))) (c 95 ()))))) 1) ((c (c 8 (c 11 ())) (c (c 20 (c 5 (c 11 ()))) ())) 4 (q . 2) (c (c (q . 1) 5) (c (a 22 (c 2 (c 11 (q 1)))) ()))) (a (i 5 (q 4 (q . 4) (c (c (q . 1) 9) (c (a 22 (c 2 (c 13 (c 11 ())))) ()))) (q . 11)) 1) 2 (i (l 5) (q 11 (q . 2) (a 30 (c 2 (c 9 ()))) (a 30 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q . 0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3) (c (q . 0x87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5) (c (q 2 (q 2 10 (c 2 (c (a 30 (c 2 (c 5 (c 11 (c () (c 23 ())))))) (c 11 ())))) (c (q ((a (i (all (not (= 9 ())) (not (= 21 ())) (not (= 45 ())) (not (= 93 ())) (not (= -67 ())) (not (= 381 ())) (not (= 765 ())) (not (= 1533 ())) (not (= 3069 ()))) (q 1 . -1) (q 2 22 (c 2 (c (c 9 (c 21 (c 45 (c 93 (c -67 (c 381 (c 765 (c 1533 (c 3069 ()))))))))) (c 11 ()))))) 1) 2 (i (all (= 5 47) (= 11 47) (= 23 47)) (q 1 . 1) ()) 1) (c (a 8 (c 2 (c 5 (c 11 ())))) (c 5 ())) (a (i (any (a 12 (c 2 (c 9 (c 21 (c 45 (c 11 ())))))) (a 12 (c 2 (c 93 (c -67 (c 381 (c 11 ())))))) (a 12 (c 2 (c 765 (c 1533 (c 3069 (c 11 ())))))) (a 12 (c 2 (c 9 (c 93 (c 765 (c 11 ())))))) (a 12 (c 2 (c 21 (c -67 (c 1533 (c 11 ())))))) (a 12 (c 2 (c 45 (c 381 (c 3069 (c 11 ())))))) (a 12 (c 2 (c 9 (c -67 (c 3069 (c 11 ())))))) (a 12 (c 2 (c 45 (c -67 (c 765 (c 11 ()))))))) (q . 11) ()) 1) 2 (i (l 5) (q 4 (a (i (= 23 47) (q 2 (i (= () 9) (q . 11) (q 8 (q . "update error"))) 1) (q . 9)) 1) (a 30 (c 2 (c 13 (c 11 (c (+ 23 (q . 1)) (c 47 ()))))))) ()) 1) 1)) (c (q 2 (q 2 (q 2 10 (c 2 (c (a 30 (c 2 (c 5 (c 11 (c () (c 23 ())))))) (c 11 ())))) (c (q ((a (i (all (not (= 9 ())) (not (= 21 ())) (not (= 45 ())) (not (= 93 ())) (not (= -67 ())) (not (= 381 ())) (not (= 765 ())) (not (= 1533 ())) (not (= 3069 ()))) (q 1 . -1) (q 2 22 (c 2 (c (c 9 (c 21 (c 45 (c 93 (c -67 (c 381 (c 765 (c 1533 (c 3069 ()))))))))) (c 11 ()))))) 1) 2 (i (all (= 5 47) (= 11 47) (= 23 47)) (q 1 . 1) ()) 1) (c (a 8 (c 2 (c 5 (c 11 ())))) (c 5 ())) (a (i (any (a 12 (c 2 (c 9 (c 21 (c 45 (c 11 ())))))) (a 12 (c 2 (c 93 (c -67 (c 381 (c 11 ())))))) (a 12 (c 2 (c 765 (c 1533 (c 3069 (c 11 ())))))) (a 12 (c 2 (c 9 (c 93 (c 765 (c 11 ())))))) (a 12 (c 2 (c 21 (c -67 (c 1533 (c 11 ())))))) (a 12 (c 2 (c 45 (c 381 (c 3069 (c 11 ())))))) (a 12 (c 2 (c 9 (c -67 (c 3069 (c 11 ())))))) (a 12 (c 2 (c 45 (c -67 (c 765 (c 11 ()))))))) (q . 11) ()) 1) 2 (i (l 5) (q 4 (a (i (= 23 47) (q 2 (i (= () 9) (q . 11) (q 8 (q . "update error"))) 1) (q . 9)) 1) (a 30 (c 2 (c 13 (c 11 (c (+ 23 (q . 1)) (c 47 ()))))))) ()) 1) 1)) (c (q () () () () () () () () ()) (c (q . 1) 1))) 1)))))) (r 1)) 1)))' '((0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba 1023) 1023 (1023 4 2))'

((70 0x3c54684107b6212d4980a23b14aca6be9e67943e1f100642c2b270a32641f915) (73 1023) (51 0x1ecfc703a3e6a33a169c59075f194e0eb0c10738d396bdc71fa14ac7099be4a9 1023))
